In [ ]:
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg

# ---------------------------------------------------------
# 1. Load data
# ---------------------------------------------------------
# Adjust the path to where your file is stored
file_path = r"../Data/GDP_forecast.csv"

df = pd.read_csv(file_path)

# Parse Date column and sort chronologically
df['Date'] = pd.to_datetime(df['Date'], dayfirst=False)  # "10/1/2025" -> 2025-10-01
df = df.sort_values('Date').reset_index(drop=True)

# ---------------------------------------------------------
# 2. Helper: rolling AR forecast function
# ---------------------------------------------------------
def rolling_ar_forecast(series, lags=1, min_obs=36):
    """
    series : pd.Series of realized GDP values
    lags   : AR order (AR(1) by default)
    min_obs : minimum number of observations before
              we start estimating AR models

    Returns a pd.Series with the same index:
    - for t < min_obs: use the original realized values
    - for t >= min_obs: use 1-step-ahead AR forecasts
    """
    series = series.astype(float)
    n = len(series)
    forecasts = pd.Series(index=series.index, dtype=float)

    for t in range(min_obs, n):
        # Use data up to t-1 for estimation
        train = series.iloc[:t]
        # Fit AR model on rolling window (min_obs..t-1)
        model = AutoReg(train, lags=lags, old_names=False).fit()
        # Forecast value at position t
        # start=end=len(train) gives 1-step-ahead forecast
        fcast = model.predict(start=len(train), end=len(train))[0]
        forecasts.iloc[t] = fcast

    # For the initial part where we can't estimate AR reliably,
    # just keep the realized values (or you can leave as NaN)
    forecasts.iloc[:min_obs] = series.iloc[:min_obs]

    return forecasts

# ---------------------------------------------------------
# 3. Apply AR forecasting to CH, JP, EU
# ---------------------------------------------------------
countries_to_forecast = ['CH', 'JP', 'EU']

for c in countries_to_forecast:
    df[c] = rolling_ar_forecast(df[c], lags=1, min_obs=36)

# ---------------------------------------------------------
# 4. Save result: same structure as original file
# ---------------------------------------------------------
output_path = r"C:\path\to\your\GDP_forecast_AR.csv"
df.to_csv(output_path, index=False)

print("Done. AR-based forecasts for CH, JP, and EU saved to:")
print(output_path)
print(df.head())
